In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [2]:
#Load the trained model
model = load_model('model.h5')

#load the encoder and scler
with open('label_gender.pkl','rb') as file:
    label_gender = pickle.load(file)

with open('onehot_geo.pkl','rb') as file:
    onehot_geo = pickle.load(file)

with open('scaler.pkl','rb') as file:
    scaler = pickle.load(file)


In [3]:
#Example input
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}
input_data

{'CreditScore': 600,
 'Geography': 'France',
 'Gender': 'Male',
 'Age': 40,
 'Tenure': 3,
 'Balance': 60000,
 'NumOfProducts': 2,
 'HasCrCard': 1,
 'IsActiveMember': 1,
 'EstimatedSalary': 50000}

In [4]:
#One hot encode 'Geography'
geo_encoded = onehot_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_geo.get_feature_names_out(['Geography']))
geo_encoded_df

d:\DL PROJECT 1\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
#encode gender column
input_data['Gender'] = label_gender.transform(input_data['Gender'])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
#Concat one hot encoded
input_df = pd.concat([input_data.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [8]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [9]:
#Scaling the input data
scaled_input=scaler.transform(input_df)
scaled_input

array([[-0.47154541,  0.90911166,  0.09477172, -0.69844549, -0.29010416,
         0.80510537,  0.63367318,  0.95214374, -0.84805047,  0.98019606,
        -0.57581067, -0.56349184]])

In [10]:
#predict churn 
pred = model.predict(scaled_input)
pred

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step


array([[0.05217747]], dtype=float32)

In [11]:
prediction_prob = pred[0][0]
prediction_prob

np.float32(0.052177474)

In [12]:
if prediction_prob>0.5:
    print("Customer is likely to churn.")
else:
    print("Customer is not likely to churn.")

Customer is not likely to churn.
